In [1]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version은 코랩에서만 동작합니다.
    %tensorflow_version 2.x
    %pip install -q -U tensorflow-addons
    %pip install -q -U transformers
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("감지된 GPU가 없습니다. GPU가 없으면 LSTM과 CNN이 매우 느릴 수 있습니다.")
    if IS_COLAB:
        print("런타임 > 런타임 유형 변경 메뉴를 선택하고 하드웨어 가속기로 GPU를 고르세요.")

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)
tf.random.set_seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

감지된 GPU가 없습니다. GPU가 없으면 LSTM과 CNN이 매우 느릴 수 있습니다.


**문장 단위 RNN(character RNN)** : 문장에서 다음 글자를 예측하도록 훈련

**상태가 없는 RNN(stateless RNN)** : 각 반복에서 무작위하게 택한 텍스트의 일부분을 학습, 나머지 텍스트에서 어떤 정보도 사용하지 않음

**상태가 있는 RNN(stateful RNN)** : 훈련 반복 사이에 은닉 상태를 유지하고 중지된 곳에서 이어서 상태를 반영

**어텐션 메커니즘(attention mechanism)** : 각 타임스텝에서 모델이 집중해야 할 입력 부분을 선택하도록 학습하는 신경망 구성 요소

**트랜스포머(transformer)** : RNN을 모두 제거하고 어텐션만 사용해 매우 좋은 성능을 내는 구조

## 16.1 Char-RNN을 사용해 셰익스피어 문체 생성하기

### 16.1.1 훈련 데이터셋 만들기
+ get_file() 함수를 사용해 셰익스피어 작품을 모두 다운로드
+ 모든 글자를 정수로 인코딩(케라스의 Tokenizer 클래스를 사용)
+ 클래스의 객체를 텍스트에 훈련(텍스트에서 사용되는 모든 글자를 찾아 각기 다른 글자ID에 매핑)

In [2]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

+ char_level = True로 지정하여 단어 수준 인코딩 대신 글자 수준 인코딩을 생성(기본적으로 텍스트를 소문자화)
+ 문장을 글자 ID로 인코딩하거나 반대로 디코딩할 수 있음
+ 이를 통해 텍스트에 있는 고유 글자 개수와 전체 글자 개수를 알 수 있음

In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [4]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [5]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [6]:
max_id = len(tokenizer.word_index) # 고유한 문자 개수
dataset_size = tokenizer.document_count # 전체 문자 개수

+ 전체 텍스트를 인코딩하여 각 글자를 ID로 나타냄(1 ~ 39 대신 0  ~38로 나타내기 위해 1을 뺌)

In [7]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

### 16.1.2 순차 데이터셋을 나누는 방법
+ 훈련 세트, 검증 세트, 테스트 세트가 중복되지 않도록 만듦
+ 시계열을 다룰 때는 보통 시간에 따라 나눔(경우에 따라 훈련할 시간 간격을 더 길게 하기 위해 다른 차원을 기준으로 나눌 수 있음)
+ 암묵적으로 RNN이 과거에서 학습하는 패턴이 미래에도 등장한다고 가정(시계열 데이터가 **변하지 않는다**고 가정)
+ 텍스트의 처음 90%를 훈련 세트로 사용(한 번에 한 글자씩 반환하는 tf.data.Dataset 객체 생성)

In [8]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

### 16.1.3 순차 데이터를 윈도 여러 개로 자르기
+ window() 메서드를 사용해 긴 시퀀스를 작은 많은 텍스트 윈도로 변환

**TBPTT(truncated backpropagation through time) :**

데이터셋의 각 샘플은 전체 텍스트에서 매우 짧은 부분 문자열, RNN은 이 부분 문자열 길이만큼만 역전파를 위해 펼쳐짐
+ window() 메서드는 윈도를 중복하지 않음(shift = 1로 지정하면 가장 큰 훈련 세트를 만들 수 있음)
+ 모든 윈도가 동일하게 101개의 글자를 포함하도록 drop_remainder=True로 지정

In [9]:
n_steps = 100
window_length = n_steps + 1 # 타깃 = 한 글자 앞선 입력
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

**중첩 데이터셋(nested dataset)** : 각각 하나의 데이터셋으로 표현되는 윈도를 포함하는 데이터셋
+ 데이터셋 메서드를 호출하여 각 윈도를 변환할 때 유용
+ but, 모델은 데이터셋이 아니라 텐서를 기대하기 때문에 훈련에 중첩 데이터셋을 바로 사용할 수 없음

**플랫 데이터셋(flat dataset)** : 데이터셋이 들어 있지 않는 데이터셋
+ flat_map() 메서드 : 중첩 데이터셋을 플랫 데이터셋으로 변환
    + 중첩 데이터셋을 평평하게 만들기 전에 각 데이터셋에 적용할 만한 함수를 매개변수로 받을 수 있음

In [10]:
# 윈도마다 batch(window_length)를 호출
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [11]:
# 윈도를 섞은 뒤 윈도를 배치로 만들과 입력과 타깃을 분리
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

+ 윈도 크기는 101 대신 11, 배치 크기는 32 대신 3으로 표현

<img src="img/16-1.png" width="500px" align='left'>  

+ 범주형 입력 특성은 원-핫 벡터나 임베딩으로 인코딩되어야 함
+ 프리페칭 추가

In [12]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [13]:
dataset = dataset.prefetch(1)

### 16.1.4 Char-RNN 모델 만들고 훈련하기
이전 글자 100개를 기반으로 다음 글자를 예측하기
+ 유닛 128개를 가진 GRU 층 2개와 입력(dropout)과 은닉 상태(recurrent_dropout)에 20% 드롭아웃을 사용
+ 출력층 : TimeDistributed 클래스를 적용한 Dense 층
+ 타임 스텝에서 출력 확률의 합은 1이어야 하므로 Dense 층의 출력에 소프트맥스 함수 적용
+ sparse_categorical_crossentropy 손실과 Adam 옵티마이저를 사용해 모델의 compile() 메서드 호출

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

### 16.1.5 Char-RNN 모델 사용하기
+ 새로운 텍스트 주입을 위한 전처리 및 다음 글자 예측

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [ ]:
X_new = preprocess(["How are yo"])
#Y_pred = model.predict_classes(X_new)
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

### 16.1.6 가짜 셰익스피어 텍스트를 생성하기
**tf.random.categorical()** : 모델이 추정한 확률을 기반으로 다음 글자를 무작위로 선택
+ 클래스의 로그 확률(로짓)을 전달하면 랜덤하게 클래스 인덱스를 샘플링함

**온도(temperature)** : 생성된 텍스트의 다양성을 더 많이 제어하기 위해 로짓을 나누는 숫자
+ 0에 가까울수록 높은 확률을 가진 글자를 선택
+ 매우 높으면 모든 글자가 동일한 확률을 가짐

다음 next_char() 함수는 이 방식을 사용해 다음 글자를 선택하고 입력 텍스트에 추가함

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

next_char() 함수를 반복 호출하여 다음 글자를 얻고 텍스트에 추가하는 작은 함수를 만듦

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

온도를 다르게 해보며 테스트

In [ ]:
tf.random.set_seed(42)
print(complete_text("t", temperature=0.2))

In [ ]:
print(complete_text("w", temperature=1))

In [ ]:
print(complete_text("w", temperature=2))

이 모델은 1에 가까운 온도에서 가장 잘 작동함

더 좋은 텍스트를 생성하려면 GRU 층과 층의 뉴런 수를 늘리고 더 오래 훈련하거나 규제를 추가

### 16.1.7 상태가 있는 RNN
**상태가 없는 RNN(stateless RNN)**
+ 훈련 반복마다 모델의 은닉 상태를 0으로 초기화
+ 타임 스텝마다 이 상태를 업데이트하고 마지막 타임 스텝 후에는 더 필요가 없기 때문에 버림

**상태가 있는 RNN(stateful RNN)**
+ RNN이 한 훈련 배치를 처리한 후에 마지막 상태를 다음 훈련 배치의 초기 상태로 사용
+ 역전파는 짧은 시퀀스에서 일어나지만 모델이 장기간 패턴을 학습할 수 있음

**상태가 있는 RNN 구성 방법**
+ 배치에 있는 각 입력 시퀀스가 이전 배치의 시퀀스가 끝난 지점에서 시작
+ 순차적이고 겹치지 않는 입력 시퀀스를 만들어야함
+ Dataset을 만들 때 window() 메서드에서 shift = n_steps를 사용(shift = 1 대신), shuffle() 메서드 호출 X
+ 상태가 있는 RNN을 위한 데이터셋은 상태가 없는 RNN보다 배치를 구성하기 더 힘듦
+ 윈도가 끝난 지점부터 다음 배치가 계속되지 않음(e.g. 1부터 32 / 33부터 64...)
+ 각 배치의 첫 번째 윈도를 생각하면 연속적이지 않음 -> 하나의 윈도를 갖는 배치를 만들기

<img src="img/16-2.png" width="500px" align='left'>  

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

+ 각 순환 층을 만들 때 stateful = True로 지정해야 함
+ 상태가 있는 RNN은 배치 크기를 알아야 함(배치에 있는 입력 시퀀스의 상태 보존)
    + 첫 번째 층에 batch_input_shape 매개변수를 지정

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

+ 에포크 끝마다 텍스트를 다시 시작하기 전에 상태를 재설정함(콜배 함수 사용)
+ 모델 컴파일하고 훈련

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])

## 16.2 감성 분석

**IMDb 리뷰 데이터셋**
+ 영어로 쓰인 영화 리뷰 50000개(25000개는 훈련, 25000개는 테스트)로 구성
+ 각 리뷰가 부정적인지(0) 긍정적인지(1) 나타내는 간단한 이진 타깃이 포함
+ 이미 전처리된 데이터셋(각 리뷰는 넘파이 정수 배열로 표현 - 각 정수는 하나의 단어)
+ 구두점을 모두 제거하고 단어는 소문자로 변환한 다음 공백으로 나누어 빈도에 따라 인덱스를 붙임(낮은 정수 = 자주 등장하는 단어)
+ 0 = 패딩 토큰
+ 1 = SOS(start-of-sequence) 토큰
+ 2 = 알 수 없는 단어

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

In [ ]:
X_train[0][:10]

In [ ]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

**다쿠 구도 - 부분 단어(subword) 수준으로 텍스트를 토큰화하거나 복원하는 비지도 학습 방법 소개**
+ 공백을 하나의 문자로 취급(언어 독립적)
+ 모델이 이전에 본 적이 없는 단어를 만나더라도 의미를 추측 가능

In [ ]:
# 텐서플로 데이터셋을 사용해 원본 IMDb 리뷰를 텍스트(바이트 스트링)로 적재
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

**전처리 함수 생성**
+ 리뷰 텍스트를 잘라내어 각 리뷰에서 처음 300 글자만 남김
+ 정규식(regular expression)을 사용해 $<br />$ 태그를 공백으로 바꿈(문자와 작은 따옴표가 아닌 다른 모든 문자를 공백으로)
+ preprocess() 함수는 리뷰를 공백으로 나눔(래그드 텐서(ragged tensor)가 반환
+ 래그드 텐서를 밀집 텐서로 바꾸고 동일한 길이가 되도록 패딩 토큰 $<pad>$으로 모든 리뷰를 패딩

In [ ]:
# 전처리 함수 생성
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

+ 어휘 사전을 구축
+ 전체 훈련 세트를 한 번 순회하면서 preprocess() 함수를 적용하고 Counter로 단어의 등장 횟수를 셈

In [ ]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [ ]:
# 가장 많이 등장하는 단어 3개 확인
vocabulary.most_common()[:3]

In [ ]:
# 어휘 사전 중에서 가장 많이 등장하는 단어 10000개만 남기고 삭제
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

+ 각 단어를 ID로 바꾸는 전처리 단계를 추가
+ 1000개의 oov(out-of-vocabulary) 버킷을 사용하는 룩업 테이블(lookup table)을 만듦

In [ ]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [ ]:
# 테이블에서 단어 몇 개에 대한 ID를 확인
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

+ 리뷰를 배치로 묶고 preprocess() 함수를 사용해 단어의 짧은 시퀀스로 바꿈
+ 앞서 만든 테이블을 사용하는 encode_words() 함수로 단어를 인코딩
+ 배치를 프리페치

In [ ]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

+ 모델을 만들어 훈련
+ 첫 번째 층 : 단어 ID를 임베딩으로 변환하는 Embedding 층
    + 단어 ID당 하나의 행과 임베딩 차원당 하나의 열을 가짐
+ 나머지 중간 층 : GRU 층 2개로 구성되고 두 번째 층은 마지막 타임 스텝의 출력만 반환
+ 출력층 : 시그모이드 활성화 함수를 사용하는 하나의 뉴런

In [ ]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

### 16.2.1 마스킹
**Embedding 층 만들 때 mask_zero = True 로 지정**
+ 패딩 토큰을 무시하도록 모델에게 알려주어 실제 의미가 있는 데이터에 집중할 수 있게 만듦
+ Embedding 층이 K.not_equal(inputs, 0)와 같은 마스크 텐서를 만듦
+ 마스크 텐서는 모델에 의해 이어지는 모든 층에 타임 스텝 차원이 유지되는 한 자동으로 전파
    + 위의 예에서는 두 번째 GRU 층이 시퀀스를 반환하지 않기 때문에 Dense 층에는 마스크 텐서가 전달 X
+ 순환 층은 마스킹된 타임 스텝을 만나면 이전 타임 스텝의 출력을 단순히 복사함
+ 마스크를 받는 모든 층은 마스킹을 지원(support_masking 속성 값이 True)
+ 마스킹을 지원하는 사용자 정의 층을 구현
    + call() 메서드에 mask 매개변수 추가
    + 생성자에서 self.supports_masking = True 로 지정
+ 마스킹 층과 마스크 자동 전파는 Sequential 모델에 가장 잘 맞음

In [ ]:
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

TensorBoard() 콜백을 사용하면 텐서보드에서 학습된 임베딩을 시각화할 수 있음

### 16.2.2 사전훈련된 임베딩 재사용하기
**모듈(module)** : 모델 컴포넌트(텐서플로 허브 프로젝트는 사전훈련된 모델 컴포넌트를 모델에 추가하기 쉽게 만들어줌)
+ hub.KerasLayer 층 : 주어진 URL에서 모듈을 다운로드

**문장 인코더(sentence encoder)** : 문자열을 입력으로 받아 하나의 벡터로 인코딩함
+ 내부적으로는 문자열을 파싱(공백으로 단어를 나눔)해서 대규모 코퍼스에서 사전훈련된 임베딩 행렬을 사용해 각 단어를 임베딩함
+ 모든 단어 임베딩의 평균을 계산함(결과값 = 문장 임베딩 : 단어 임베딩의 평균과 문장에 있는 단어 수의 제곱근을 곱한 것)
+ 두 개의 Dense 층을 추가해 감성 분석 모델을 만듦(기본적으로 hub.KerasLayer 층은 훈련되지 않음)

In [ ]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

IMDb 리뷰 데이터셋을 다운로드하고 배치, 프리페치 이후 모델을 훈련

In [ ]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

## 16.3 신경망 기계 번역을 위한 인코더-디코더 네트워크

**영어 문장을 프랑스어로 번역하는 신경망 기계 번역 모델(neural machine translation, NMT)**
+ 영어 문장을 인코더로 주입하면 디코더는 프랑스어 번역을 출력
+ 디코더는 이전 스텝에서 출력된 단어를 입력으로 사용
+ 맨 처음 단어는 SOS(start-of-sequence) 토큰으로 시작 / 디코더는 문장의 끝에 EOD(end-of-sequence) 토큰이 있을 것으로 기대
+ 영어 문장은 인코더로 주입되기 전에 거꾸로 뒤집힘 / 즉, 영어 문장의 시작 부분을 인코더에 마지막으로 주입('I drink milk' -> 'milk drink I')
+ 각 단어는 초기에 1차원으로 표현 -> 임베딩 층이 단어 임베딩을 반환 -> 단어 임베딩이 인코더와 디코더로 주입

<img src="img/16-3.png" width="500px" align='left'>  

+ 각 단계마다 디코더는 출력 어휘 사전(프랑스어)에 있는 단어에 대한 점수를 출력
+ 소프트맥스 층이 이 점수를 확률로 바꿈
+ 가장 높은 확률의 단어가 출력
+ Char-RNN 모델에서 했던 것처럼 "sparse_categorical_crossentropy" 손실 함수를 사용해 훈련
+ 훈련 이후 추론 시에는 디코더에 주입할 타깃 문장이 없음(대신 이전 스텝에서 디코더가 출력한 단어를 주입)

<img src="img/16-4.png" width="500px" align='left'>  

+ 문장을 비슷한 길이의 버킷으로 그룹핑 / 버킷에 담긴 문장이 모두 동일한 길이가 되도록 패딩을 추가
+ EOS 토큰 이후 출력은 모두 무시(손실에 영향을 미치지 않음
+ **샘플링 소프트맥스(sampled softmax)**
    + tf.nn.sampled_softmax_loss() : 단어에 대한 로짓과 타깃이 아닌 단어 중 무작위로 샘플링한 단어의 로짓만 고려

In [ ]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

+ LSTM 층을 만들 때 최종 은닉 상태를 디코더로 보내기 위해 return_state = True로 지정
+ LSTM 셀을 사용하기 때문에 은닉 상태 두 개(장기, 단기)를 반환
+ TrainingSampler은 각 스텝에서 디코더에게 이전 스텝의 출력이 무엇인지 알려줌
+ 추론 시에는 실제로 출력되는 토큰의 임베딩이 됨 / 훈련 시에는 이전 타깃 토큰의 임베딩이 됨

### 16.3.1 양방향 RNN
+ 일반 순환 층은 과거와 현재의 입력만 보고 출력을 생성

**양방향 순환 층(bidirectional recurrent layer)**
+ 동일한 입력에 대해 두 개의 순환 층을 실행(하나는 왼쪽에서 오른쪽으로, 다른 하나는 오른쪽에서 왼쪽으로)
+ 일반적으로 타임 스텝마다 이 두 출력을 연결
+ keras.layers.Bidirectional으로 순환 층을 감싸기

<img src="img/16-5.png" width="500px" align='left'>  

In [ ]:
# 양방향 GRU 층 만들기
model = keras.models.Sequential([
    keras.layers.GRU(10, return_sequences=True, input_shape=[None, 10]),
    keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))
])

model.summary()

### 16.3.2 빔 검색
**빔 검색(beam search)**
+ k개의 가능성 있는 문장의 리스트를 유지하고 디코더 단계마다 이 문장의 단어를 하나씩 생성하여 가능성 있는 k개의 문장을 만듦
+ 파라미터 k : 빔 너비(beam width)

## 16.4 어텐션 메커니즘

**바흐다나우 어텐션(Bahdanau attention) / 연결 어텐션(concatenative attention) / 덧셈 어텐션(additive attention)**
+ 각 타임 스텝에서 적절한 단어에 디코더가 초점을 맞추도록 하는 기술 소개
+ RNN의 단기 기억의 제한성에 훨씬 적은 영향을 받으며 긴 문장에 대해서 최고 수준의 성능을 크게 향상시킴

**모델 구조**
+ 왼쪽에 인코더와 디코더가 있음
+ 인코더의 마지막 은닉 상태만 디코더에 보내는 것이 아니라 인코더의 모든 출력을 디코더로 전송
+ 각 타임 스텝에서 디코더의 메모리 셀은 이런 모든 인코더 출력의 가중치 합을 계산
+ $\alpha_{(t,j)}$ : $t$번째 디코더 타임 스텝에서 $i$번째 인코더 출력의 가중치
+ 타임 스텝마다 메모리 셀이 앞서 언급한 입력과 이전 타임 스텝의 은닉 상태를 받음
+ 마지막으로 이전 타임 스텝에서 타깃 단어를 받음

<img src="img/16-6.png" width="500px" align='left'>

+ $\alpha_{(t,j)}$ 가중치는 정렬 모델(alignment model) or 어텐션 층(attention layer)이라 부르는 작은 신경망에 의해 생성

**정렬 모델(alignment model) or 어텐션 층(attention layer)**
+ 인코더-디코더 모델의 나머지 부분과 함께 훈련
+ TimeDistribued 클래스를 적용한 Dense 층으로 시작
    + 하나의 뉴런으로 구성되고 인코더의 모든 출력을 입력으로 받아 디코더의 이전 은닉 상태를 연결
    + 각 인코더 출력에 대한 점수(또는 에너지)를 출력
    + 점수는 각 출력이 디코더의 은닉 상태와 얼마나 잘 맞는지를 측정
    + 모든 점수가 소프트맥스 층을 통과해 각 인코더 출력에 대한 최종 가중치를 얻음

### 16.4.1 비주얼 어텐션
**비주얼 어텐션(visual attention)을 사용한 이미지 캡션 생성**
+ 합성곱 신경망이 먼저 이미지를 처리하여 일련의 특성 맵을 출력
+ 어텐션 메커니즘을 장착한 디코더 RNN이 한 번에 한 단어씩 캡션을 생성
+ 디코더 타임 스텝마다 디코더는 어텐션 모델을 사용해 이미지에서 적절한 부위에 초점을 맞춤
+ 디코더가 frisbee를 출력할 때 입력 이미지의 어떤 부분에 초점을 맞추고 있는지 볼 수 있음

<img src="img/16-7.png" width="500px" align='left'>

**설명 가능성(explainability)** : 모델이 어떤 출력을 만들도록 이끄는 것이 무엇인지 이해하기 쉬움(어텐션 메커니즘의 장점)

### 16.4.2 트랜스포머 구조 : 어텐션이 필요한 전부다
**트랜스포머(transformer)**
+ 순환 층이나 합성곱 층을 전혀 사용하지 않고 어텐션 메커니즘만(그리고 임베딩 층, 밀집 층, 정규화 층..) 사용해 NMT 문제에서 최고 수준 성능 향상
+ 구조를 훨씬 빠르게 훈련할 수 있고 병렬화하기 쉬움

<img src="img/16-8.png" width="500px" align='left'>

**왼쪽(인코더)**
+ 단어 ID의 시퀀스로 표현된 문장의 배치를 입력으로 받음(입력 크기 = [배치 크기, 입력 문장의 최대 길이])
+ 인코더는 각 단어를 512차원의 표현으로 인코딩함(출력 크기 = [배치 크기, 입력 문장의 최대 길이, 512])
+ 인코더의 윗부분(위치 인코딩 다음 부분)은 N번 반복되어 쌓아 올림

**오른쪽(디코더)**
+ 훈련하는 동안 타깃 문장을 입력으로 받음(입력은 오른쪽으로 한 스텝 이동되어 있음(시작 부분에 SOS 토큰이 추가되어 있음))
+ 인코더의 출력을 받음
+ 디코더의 윗부분도 N번 반복되어 쌓아 올림
+ 인코더의 최종 출력이 N번의 디코더에 모두 주입
+ 타임 스텝마다 디코더는 가능한 다음 단어에 대한 확률을 출력(출력 크기 = [배치 크기, 출력 문장의 최대 길이, 어휘 사전 길이])

+ 추론 시에는 디코더에 타깃을 주입할 수 없음 -> 이전 타임 스텝에서 출력된 단어를 주입
+ 이 모델이 반복적으로 실행되어 매번 하나의 단어를 예측
+ 임베딩 층 2개, 스킵 연결 5 X N개, 정규화 층, 밀집 층 2개(첫 번째는 Relu 활성화 함수, 두 번째는 소프트맥스 함수)로 구성된 피드포워드 모듈이 2 X N개
+ 출력층은 소프트맥스 활성화 함수를 사용하는 밀집 층
+ 모든 층은 타임 스텝에 독립적(time-distributed)

**멀티-헤드 어텐션(multi-head-attention) 층** : 관련이 많은 단어에 더 많은 주의를 기울이면서 각 단어와 동일한 문장에 있는 다른 단어의 관계를 인코딩
+ 디코더의 위쪽 멀티-헤드-어텐션 층은 디코더가 입력 문장에 있는 단어에 주의를 기울이는 곳

**셀프-어텐션(self-attention)** : 문장 자기 자신에게 주의를 기울임

**마스크드 멀티-헤드 어텐션(masked multi-head-attention) 층** : 동일한 작업을 수행하지만 각 단어는 이전에 등장한 단어에만 주의를 기울임

**위치 인코딩(positional encoding)** : 문장에 있는 단어의 위치를 나타내는 단순한 밀집 벡터
+ 모델을 사용해 위치 인코딩을 학습할 수 있지만 논문에서 저자들은 여러 가지 주기의 사인과 코사인 함수로 정의한 고정된 위치 인코딩을 선호
+ n번째 위치 인코딩이 각 문자에 있는 n번째 단어의 단어 임베딩에 더해짐 -> 모델이 각 단어의 위치를 알 수 있음
+ 상대적이고 절대적인 단어 위치가 중요하므로 이 정보를 트랜스포머에게 어떻게든 전달해야 함

### 위치 인코딩
+ 위치마다 고유한 위치 인코딩이 만들어지기 때문에 위치 인코딩을 단어 임베딩에 더하면 모델이 문장에 있는 단어의 절대 위치를 알 수 있음
+ 진동 함수 선택에 따라 모델이 상대적인 위치도 학습할 수 있음

<img src="img/16-9.png" width="500px" align='left'>

생성자에서 위치 인코딩 행렬을 미리 계산(문장의 최대 길이 max_steps와 각 단어를 표현할 차원수 max_dims를 알아야 함)

call() 메서드에서 이 인코딩 행렬을 입력의 크기로 잘라 입력에 더함

In [ ]:
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 == 1: max_dims += 1 # max_dims must be even
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000**(2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000**(2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))
    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

트랜스포머의 첫 번째 층을 만듦

In [ ]:
embed_size = 512; max_steps = 500; vocab_size = 10000
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)
positional_encoding = PositionalEncoding(max_steps, max_dims=embed_size)
encoder_in = positional_encoding(encoder_embeddings)
decoder_in = positional_encoding(decoder_embeddings)

### 멀티-헤드 어텐션
**스케일드 점-곱 어텐션(scaled dot-product attention) 층**
+ 모델은 키를 표현하기 위한 구분되는 토큰을 갖지 않음(벡터 표현으로 이 개념을 가짐) -> 룩업에 사용할 키(쿼리)는 딕셔너리 키와 완벽 매칭 X
+ 쿼리와 딕셔너리에 있는 각 키 사이의 유사도를 계산하고 소프트맥스 함수를 사용해 유사도 점수를 합해서 1이 되는 가중치로 바꿈
+ 모델이 키에 해당하는 값의 가중치 합을 계산

<img src="img/식 16-3.png" width="250px" align='left'>

+ $Q$ : 행마다 쿼리 하나를 담은 행렬
+ $K$ : 행마다 키 하나를 담은 행렬
+ $V$ : 행마다 값 하나를 담은 행렬
+ $QK^T$ : 쿼리/키 쌍마다 하나의 유사도 점수를 담음

+ 인코더에서 위 식이 배치에 있는 모든 입력 문장에 적용($Q$, $K$, $V$는 모두 입력 문장에 있는 단어의 목록과 동일)
+ 디코더의 마스킹된 어텐션 층에서 이 식이 배치에 있는 모든 타깃 문장에 적용($Q$, $K$, $V$는 타깃 문장에 있는 단어의 목록과 동일)
+ 뒤에 오는 단어를 비교하지 않기 위해 미래의 출력 토큰을 마스킹 처리(추론할 떄 디코더는 미래 단어가 아니라 이미 출력된 단어만 참조 가능)
+ keras.layers.Attention 층은 배치에 있는 여러 문장에 식을 효율적으로 적용하는 스케일드 점-곱 어텐션을 구현

In [ ]:
Z = encoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True)([Z, Z])

encoder_outputs = Z
Z = decoder_in
for N in range(6):
    Z = keras.layers.Attention(use_scale=True, causal=True)([Z, Z])
    Z = keras.layers.Attention(use_scale=True)([Z, encoder_outputs])

outputs = keras.layers.TimeDistributed(
    keras.layers.Dense(vocab_size, activation="softmax"))(Z)

+ use_scale = True로 지정하면 파라미터가 추가되어 유사도 점수의 스케일을 적절히 낮추는 방법을 배움
+ 두 번째 어텐션 층을 만들 때 casual = True로 지정하면 각 출력 토큰은 미래 토큰이 아니라 이전 출력 토큰에만 주의를 기울임

<img src="img/16-10.png" width="500px" align='left'>

+ 멀티-헤드 어텐션은 스케일드 점-곱 어텐션 층의 묶음
+ 각 층은 값, 키, 쿼리의 선형 변환이 선행되고 출력은 단순히 모두 연결되어 마지막 선형 변환을 통과
+ 모델이 단어 표현을 여러 부분 공간(subspace)으로 다양하게 투영(부분 공간은 단어의 일부 특징에 주목)
+ 스케일드 점-곱 어텐션 층이 룩업 단계를 구현하고 마지막으로 모든 결과를 연결하여 원본 공간으로 다시 투영함

## 16.5 언어 모델 분야의 최근 혁신